In [51]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("../libs")
import pandas as pd
from data_manager.etf_data_manager import etf_data_iter, save_res_df_to_windows, get_etf_data_by_symbols
from core.portfolios.portfolio import Portfolio, Position
from config import DataPath
import datetime
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from factors.ma import SlopeRiskRatio
from factors.new_high import NewHigh
from factors.portfolio.correlation import CorrelationFactor
from factors.average_true_range import AverageTrueRange

In [3]:
from fetcher.etf import get_all_etf_code
all_codes = get_all_etf_code()


In [4]:
name_dict = all_codes[["代码", "名称"]].set_index("代码").to_dict().get("名称", {})

In [52]:
my_portfolio = Portfolio(312240.33)

In [53]:
my_portfolio.add_position_with_symbol_quantities(
    ("159206", 25000),
    ("516570", 34000),
    ("159326", 21500),
    ("560280", 19100),
    ("159201", 25300),
    ("560860", 22000),
    ("159697", 25800),
    ("159516", 15000),
    ("513750", 18000),
    ("513800", 17800)
)

In [17]:
def risk_concentration(corr_matrix):
    """风险集中度分析"""
    # 平均相关系数（衡量整体相关性）
    avg_corr = corr_matrix.values[np.triu_indices(len(corr_matrix), 1)].mean()
    
    # 相关性矩阵的特征值分析（衡量多元化程度）
    eigenvalues = np.linalg.eigvalsh(corr_matrix)
    eigenvalue_ratio = eigenvalues.max() / eigenvalues.sum()
    
    # 有效组合数（衡量真正独立的资产数量）
    if eigenvalues.sum() > 0:
        effective_n = (eigenvalues.sum()**2) / (eigenvalues**2).sum()
    else:
        effective_n = 0
    
    return {
        'average_correlation': avg_corr,
        'dominant_eigenvalue_ratio': eigenvalue_ratio,
        'effective_diversification_number': effective_n,
        'portfolio_concentration_score': eigenvalue_ratio * avg_corr
    }

In [18]:
def get_k_ratio_res(k):
    all_res = []
    for df in etf_data_iter():
        df.add_factors(SlopeRiskRatio())
        df.name = name_dict.get(df.symbol, "")
        res = df.calc_factors().tail(1)["SlopeRiskRatio"].values[0]
        all_res.append([df.symbol, df.name,res])
    save_res_df_to_windows(pd.DataFrame(all_res, columns=["code", "name", "ratio"]).sort_values(by="ratio", ascending=False), 
                           f"斜率风险比/{datetime.date.today().strftime('%Y%m%d')}-{k}倍.xlsx")

In [19]:
def generate_new_high_list(portfolio):
    add_to_list = []
    sell_list = []
    today_dir = os.path.join(DataPath.DEFAULT_WINDOWS_PATH, "新高", datetime.datetime.today().strftime("%Y%m%d"))
    if not os.path.exists(today_dir):
        os.mkdir(today_dir)
    for data in etf_data_iter():
        data.add_factors(NewHigh())
        data.calc_factors()
        data.name = name_dict[data.symbol]
        if 1 in data.factor_results["NewHigh"].iloc[-1:].values:
            add_to_list.append(data.symbol)
            data.output_with_factors_to(today_dir)
        elif portfolio.has_symbol(data.symbol):
            data.output_with_factors_to(today_dir)
            if data.factor_results["NewHigh"].iloc[-1] == -1:
                sell_list.append(data.symbol)
    if sell_list:
        pd.DataFrame(sell_list, columns=["symbol"]).to_csv(os.path.join(today_dir, "sell.csv"), index=False, encoding="utf-8-sig")
    pd.DataFrame(add_to_list, columns=["symbol"]).to_csv(os.path.join(today_dir, "add.csv"), index=False, encoding="utf-8-sig")
    return add_to_list

In [10]:
def analysis_corr_between_add_and_hold(add_to, hold):
    hold_datas = get_etf_data_by_symbols(hold)
    add_datas = get_etf_data_by_symbols([i[0] for i in add_to])
    idx = []
    corr_analysis = []
    for num, add in enumerate(add_datas):
        idx.append(add.symbol)
        corr = CorrelationFactor()
        corrs_res = risk_concentration(corr(*([hold_data for hold_data in hold_datas] + [add])))
        corrs_res["name"] = add_to[num][1]
        corr_analysis.append(corrs_res)
    final_analysis_res = pd.DataFrame(corr_analysis, index=idx).sort_values(by="effective_diversification_number", ascending=False)
    today_dir = os.path.join(DataPath.DEFAULT_WINDOWS_PATH, "新高", datetime.datetime.today().strftime("%Y%m%d"))
    final_analysis_res.to_excel(os.path.join(today_dir, "corr_analysis.xlsx"), index=True)
    return final_analysis_res

In [11]:
get_k_ratio_res(1)

  0%|          | 0/1328 [00:00<?, ?it/s]

In [21]:
add_to_list = generate_new_high_list(my_portfolio)

  0%|          | 0/1328 [00:00<?, ?it/s]

In [55]:
corr_add = my_portfolio.calc_corrs_with_current_position(add_to_list, name_dict=name_dict)
corr_add = corr_add.join(pd.Series([my_portfolio.max_money_for_symbol_by_ATR(i) for i in corr_add.index], index=corr_add.index, name="max_value"))
corr_add.to_excel(
    os.path.join(DataPath.DEFAULT_WINDOWS_PATH, "新高", datetime.datetime.today().strftime("%Y%m%d"), "add_with_corr.xlsx"), 
    index=True
)